# Library Overview

## Understanding the nanohubpadre Architecture

This notebook introduces the core components of the `nanohubpadre` library and explains how to build simulations from scratch or use the convenient device factory functions.

**Learning Objectives:**
- Understand the library architecture
- Learn about core components (Mesh, Region, Electrode, Doping, etc.)
- Build a simple simulation manually
- Use device factory functions for quick setup

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## 1. Library Architecture

The `nanohubpadre` library provides a hierarchical object model that mirrors the PADRE input file structure:

```
Simulation
    |
    +-- Mesh           # Computational grid
    +-- Region(s)      # Material regions
    +-- Electrode(s)   # Electrical contacts
    +-- Doping(s)      # Doping profiles
    +-- Contact(s)     # Contact properties
    +-- Material(s)    # Material parameters
    +-- Models         # Physical models (SRH, mobility, etc.)
    +-- System         # Solver configuration
    +-- Solve(s)       # Solution steps
    +-- Log            # Output logging
    +-- Plot1D/Plot3D  # Visualization commands
```

In [ ]:
# Import core components
from nanohubpadre import (
    # Core simulation container
    Simulation,
    
    # Device structure components
    Mesh,
    Region,
    Electrode,
    Doping,
    Contact,
    Material,
    
    # Physical models and solver
    Models,
    System,
    Method,
    Solve,
    
    # Output and visualization
    Log,
    Plot1D,
    Plot3D
)

print("All components imported successfully!")

---

## 2. Core Components Explained

### 2.1 Mesh

The mesh defines the computational grid. PADRE uses a 2D rectangular mesh with:
- `nx`: Number of nodes in x-direction
- `ny`: Number of nodes in y-direction
- Mesh spacing can be non-uniform using `ratio` for grading

In [ ]:
# Create a mesh: 100 x 20 nodes
mesh = Mesh(nx=100, ny=20)

# Define x-coordinates: 0 to 2 microns
mesh.add_x_mesh(node=1, location=0.0)      # Start at x=0
mesh.add_x_mesh(node=50, location=1.0, ratio=0.9)  # Finer mesh toward center
mesh.add_x_mesh(node=100, location=2.0, ratio=1.1) # Coarser toward end

# Define y-coordinates: 0 to 1 micron
mesh.add_y_mesh(node=1, location=0.0)
mesh.add_y_mesh(node=20, location=1.0)

print("Mesh created:")
print(f"  Size: {mesh.nx} x {mesh.ny} nodes")
print(f"  Domain: 0-2 um (x) x 0-1 um (y)")

### 2.2 Region

Regions define areas of specific materials. Each region:
- Has a unique number
- Specifies material type (silicon, oxide, etc.)
- Defines index bounds (ix_low, ix_high, iy_low, iy_high)

In [ ]:
# Define a silicon region covering the entire mesh
silicon_region = Region(
    number=1,
    ix_low=1, ix_high=100,
    iy_low=1, iy_high=20,
    silicon=True  # Or use material="silicon"
)

print("Region defined:")
print(f"  Material: Silicon")
print(f"  Bounds: ({silicon_region.ix_low}, {silicon_region.iy_low}) to ({silicon_region.ix_high}, {silicon_region.iy_high})")

### 2.3 Electrode

Electrodes define where electrical contacts are placed on the device.

In [ ]:
# Left electrode (anode)
anode = Electrode(
    number=1,
    ix_low=1, ix_high=1,    # At x=0 (left edge)
    iy_low=1, iy_high=20    # Full height
)

# Right electrode (cathode)
cathode = Electrode(
    number=2,
    ix_low=100, ix_high=100,  # At x=2um (right edge)
    iy_low=1, iy_high=20
)

print("Electrodes defined:")
print(f"  Electrode 1 (Anode): Left edge")
print(f"  Electrode 2 (Cathode): Right edge")

### 2.4 Doping

Doping profiles define the carrier concentrations. Types include:
- **Uniform**: Constant concentration in a region
- **Gaussian**: Gaussian profile with peak and junction depth

In [ ]:
# P-type doping (left half)
p_doping = Doping(
    uniform=True,
    p_type=True,
    concentration=1e17,  # 1e17 cm^-3
    x_left=0, x_right=1.0,
    y_top=0, y_bottom=1.0
)

# N-type doping (right half)
n_doping = Doping(
    uniform=True,
    n_type=True,
    concentration=1e17,
    x_left=1.0, x_right=2.0,
    y_top=0, y_bottom=1.0
)

print("Doping profiles defined:")
print(f"  P-type: 1e17 cm^-3 (x: 0-1 um)")
print(f"  N-type: 1e17 cm^-3 (x: 1-2 um)")

### 2.5 Contact

Contacts define the electrical properties of electrodes:
- **Neutral (Ohmic)**: Ideal ohmic contact
- **Schottky**: Metal-semiconductor barrier with workfunction

In [ ]:
# Ohmic contacts for all electrodes
ohmic_contact = Contact(all_contacts=True, neutral=True)

print("Contact: Ohmic (neutral) for all electrodes")

### 2.6 Models and System

**Models** enables physical effects:
- `srh`: Shockley-Read-Hall recombination
- `auger`: Auger recombination
- `conmob`: Concentration-dependent mobility
- `fldmob`: Field-dependent mobility
- `bgn`: Band-gap narrowing
- `impact`: Impact ionization

**System** configures the solver:
- `electrons`, `holes`: Carrier types to solve
- `newton`: Use Newton's method

In [ ]:
# Physical models
models = Models(
    temperature=300,    # Room temperature (K)
    srh=True,          # SRH recombination
    conmob=True,       # Concentration-dependent mobility
    fldmob=True        # Field-dependent mobility
)

# Solver system
system = System(
    electrons=True,
    holes=True,
    newton=True        # Coupled Newton solver
)

print("Models: SRH, concentration & field-dependent mobility")
print("System: 2-carrier Newton solver")

### 2.7 Solve

Solve statements define what to compute:
- `initial=True`: Equilibrium solution
- `project=True`: Project from previous solution
- Voltage sweeps with `vstep` and `nsteps`

In [ ]:
# Equilibrium solution
eq_solve = Solve(initial=True, outfile="equilibrium")

# Forward bias sweep: 0 to 0.8V in 0.1V steps
fwd_solve = Solve(
    project=True,
    v2=0.0,          # Start voltage on electrode 2
    vstep=0.1,       # Voltage step
    nsteps=8,        # Number of steps
    electrode=2,     # Sweep electrode 2
    outfile="forward"
)

print("Solve statements:")
print("  1. Equilibrium (0V)")
print("  2. Forward bias: 0 to 0.8V")

---

## 3. Building a Complete Simulation

Let's assemble all components into a complete PN diode simulation:

In [ ]:
# Create the simulation container
sim = Simulation(title="Manual PN Diode")

# 1. Define the mesh
sim.mesh = Mesh(nx=100, ny=10)
sim.mesh.add_x_mesh(1, 0.0)
sim.mesh.add_x_mesh(50, 1.0, ratio=0.9)
sim.mesh.add_x_mesh(100, 2.0, ratio=1.1)
sim.mesh.add_y_mesh(1, 0.0)
sim.mesh.add_y_mesh(10, 0.5)

# 2. Define regions
sim.add_region(Region(1, ix_low=1, ix_high=100, iy_low=1, iy_high=10, silicon=True))

# 3. Define electrodes
sim.add_electrode(Electrode(1, ix_low=1, ix_high=1, iy_low=1, iy_high=10))
sim.add_electrode(Electrode(2, ix_low=100, ix_high=100, iy_low=1, iy_high=10))

# 4. Define doping
sim.add_doping(Doping(uniform=True, p_type=True, concentration=1e17,
                      x_left=0, x_right=1.0, y_top=0, y_bottom=0.5))
sim.add_doping(Doping(uniform=True, n_type=True, concentration=1e17,
                      x_left=1.0, x_right=2.0, y_top=0, y_bottom=0.5))

# 5. Define contacts
sim.add_contact(Contact(all_contacts=True, neutral=True))

# 6. Set up models and solver
sim.models = Models(temperature=300, srh=True, conmob=True, fldmob=True)
sim.system = System(electrons=True, holes=True, newton=True)

# 7. Add solve commands
sim.add_solve(Solve(initial=True, outfile="eq"))

# 8. Add I-V logging and voltage sweep
sim.add_log(Log(ivfile="iv_data"))
sim.add_solve(Solve(project=True, v2=0.0, vstep=0.05, nsteps=16, electrode=2, outfile="fwd"))

print("Simulation built successfully!")
print("\nGenerated PADRE deck:")
print("="*60)
print(sim.generate_deck())

---

## 4. Using Device Factory Functions

While building simulations manually gives full control, the library provides **factory functions** for common devices that handle all the setup automatically.

In [ ]:
# Import device factories
from nanohubpadre import (
    create_pn_diode,
    create_schottky_diode,
    create_mosfet,
    create_bjt,
    create_solar_cell,
    create_mos_capacitor,
    create_mesfet
)

print("Device factory functions available!")

In [ ]:
# Create the same PN diode with the factory function - much simpler!
sim_factory = create_pn_diode(
    length=2.0,
    p_doping=1e17,
    n_doping=1e17,
    log_iv=True,
    forward_sweep=(0.0, 0.8, 0.05)  # (start, end, step)
)

print("Factory-created simulation:")
print("="*60)
print(sim_factory.generate_deck())

### Factory Function Parameters

Each factory function accepts parameters for:

1. **Geometry**: Device dimensions
2. **Mesh**: Grid resolution
3. **Doping**: Concentration levels
4. **Models**: Physical effects to include
5. **Output**: Logging options
6. **Sweeps**: Voltage sweep configurations

In [ ]:
# Example: MOSFET with transfer characteristic
mosfet_sim = create_mosfet(
    # Geometry
    channel_length=0.1,      # 100nm gate length
    gate_oxide_thickness=0.005,  # 5nm oxide
    
    # Doping
    channel_doping=1e18,
    source_drain_doping=1e20,
    device_type="nmos",
    
    # Output and sweep
    log_iv=True,
    vgs_sweep=(0.0, 1.5, 0.1),  # Gate voltage sweep
    vds=0.1                      # Fixed drain voltage
)

print("MOSFET simulation configured for Id-Vg characteristic")
print(f"  Gate sweep: 0V to 1.5V")
print(f"  Drain bias: 0.1V")

---

## 5. Output and Visualization

### 5.1 Band Diagram Logging

Use `log_band_diagram()` to output band structure data:

In [ ]:
# Create simulation with band diagram logging
sim_bands = create_pn_diode(
    log_bands_eq=True,    # Log bands at equilibrium
    log_bands_bias=True,  # Log bands during sweep
    forward_sweep=(0.0, 0.6, 0.2)
)

print("Band diagram logging enabled")
print("Output files will include:")
print("  - cbeq (conduction band at equilibrium)")
print("  - vbeq (valence band at equilibrium)")
print("  - cbfwd, vbfwd (bands at forward bias)")

### 5.2 I-V Logging

Enable I-V data collection with `log_iv=True`:

In [ ]:
# I-V logging example
sim_iv = create_pn_diode(
    log_iv=True,
    iv_file="diode_iv",  # Output filename
    forward_sweep=(0.0, 0.8, 0.05)
)

print("I-V logging enabled")
print("Data will be saved to: diode_iv")

### 5.3 Plot Commands

Add custom plot commands for specific quantities:

In [ ]:
# Add custom plot commands
sim_custom = create_pn_diode()
sim_custom.add_solve(Solve(initial=True))

# Plot doping profile
sim_custom.add_command(Plot1D(
    doping=True,
    x_start=0, x_end=2.0,
    y_start=0.25, y_end=0.25,  # Horizontal cut
    outfile="doping_profile",
    ascii=True
))

# Plot electric field
sim_custom.add_command(Plot1D(
    e_field=True,
    x_start=0, x_end=2.0,
    y_start=0.25, y_end=0.25,
    outfile="efield",
    ascii=True
))

print("Custom plot commands added:")
print("  - Doping profile")
print("  - Electric field")

---

## 6. Running Simulations

To execute a simulation, use the `run()` method:

In [ ]:
# Example of running a simulation (requires PADRE to be installed)
sim_run = create_pn_diode(
    log_iv=True,
    log_bands_eq=True,
    forward_sweep=(0.0, 0.6, 0.1)
)

# Generate the deck (always works)
deck = sim_run.generate_deck()
print("PADRE input deck generated successfully!")
print(f"Deck length: {len(deck)} characters")

# To run the simulation (uncomment if PADRE is installed):
# result = sim_run.run()
# print(f"Simulation completed with return code: {result.returncode}")

---

## Summary

In this notebook, you learned:

1. **Library Architecture**: How the components fit together
2. **Core Components**: Mesh, Region, Electrode, Doping, Contact, Models, System, Solve
3. **Manual Construction**: Building simulations step-by-step
4. **Factory Functions**: Quick device setup with `create_*()` functions
5. **Output Options**: Band diagrams, I-V logging, custom plots

**Next Steps**: 
- [02 - PN Diode](02_PN_Diode.ipynb): Deep dive into PN junction physics and simulation